# Recurring Neural Networks with Keras

## Sentiment analysis from movie reviews

The data set consists of user-generated movie reviews and classification of whether the user liked the movie or not based on its associated rating.

More info on the dataset is here:
https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

Use an RNN to do sentiment analysis on full-text movie reviews. Train an artificial neural network how to "read" movie reviews and guess whether the author liked the movie or not from them.

Since understanding written language requires keeping track of all the words in a sentence, need a recurrent neural network to keep a "memory" of the words that have come before as it "reads" sentences over time.

In particular, use LSTM (Long Short-Term Memory) cells because don't really want to "forget" words too quickly - words early on in a sentence can affect the meaning of that sentence significantly.

Importing the stuff we need:

In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

Import our training and testing data. (20,000 most popular words in the dataset in order to keep things somewhat managable.) The dataset includes 5,000 training reviews and 25,000 testing reviews for some reason.

In [2]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

Loading data...


Look at the first training feature, which should represent a written movie review:

In [3]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

This data set has spared a lot of trouble - they have already converted words to integer-based indices. The actual letters that make up a word don't really matter, what matters are the words themselves - and the model needs numbers to work with, not letters.

Each number in the training features represent some specific word. 

What do the labels look like?

In [4]:
y_train[0]

1

Just 0 or 1, which indicates whether the reviewer said they liked the movie or not.

There are movie reviews that have been converted into vectors of words represented by integers, and a binary sentiment classification to learn from.

RNN's can blow up quickly, so again to keep things managable on our little PC let's limit the reviews to their first 80 words:

In [5]:
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

Start with an Embedding layer (converts the input data into dense vectors of fixed size that's better suited for a neural network.) The 20,000 indicates the vocabulary size (only want the top 20,000 words) and 128 is the output dimension of 128 units.

Set up a LSTM layer for the RNN itself. Specify 128 to match the output size of the Embedding layer, and dropout terms to avoid overfitting, which RNN's are particularly prone to.

Boil it down to a single neuron with a sigmoid activation function to choose our binary sentiment classification of 0 or 1.

In [6]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Use the binary_crossentropy loss function. And the Adam optimizer is usually a good choice.

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Train our model. RNN's, like CNN's, are very resource heavy. Keeping the batch size relatively small is the key to enabling this to run on PC at all.

Training:

In [9]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=15,
          verbose=2,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 - 83s - loss: 0.4605 - accuracy: 0.7798 - val_loss: 0.3795 - val_accuracy: 0.8349
Epoch 2/15
25000/25000 - 80s - loss: 0.3014 - accuracy: 0.8780 - val_loss: 0.4075 - val_accuracy: 0.8237
Epoch 3/15
25000/25000 - 81s - loss: 0.2124 - accuracy: 0.9182 - val_loss: 0.4114 - val_accuracy: 0.8238
Epoch 4/15
25000/25000 - 81s - loss: 0.1520 - accuracy: 0.9445 - val_loss: 0.4581 - val_accuracy: 0.8271
Epoch 5/15
25000/25000 - 79s - loss: 0.1097 - accuracy: 0.9603 - val_loss: 0.5656 - val_accuracy: 0.8252
Epoch 6/15
25000/25000 - 78s - loss: 0.0772 - accuracy: 0.9724 - val_loss: 0.6683 - val_accuracy: 0.8210
Epoch 7/15
25000/25000 - 79s - loss: 0.0622 - accuracy: 0.9790 - val_loss: 0.7128 - val_accuracy: 0.8209
Epoch 8/15
25000/25000 - 81s - loss: 0.0366 - accuracy: 0.9877 - val_loss: 0.9249 - val_accuracy: 0.7985
Epoch 9/15
25000/25000 - 79s - loss: 0.0291 - accuracy: 0.9912 - val_loss: 0.9049 - val_accuracy: 0.8130
Epoch

Evaluate model's accuracy:

In [10]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=32,
                            verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

25000/1 - 14s - loss: 1.3717 - accuracy: 0.8053
Test score: 1.144708181578219
Test accuracy: 0.80528


80% accuracy, considering limited to just the first 80 words of each review.

Note that the validation accuracy while training never really improved after the first epoch; likely just overfitting. This is a case where early stopping would have been beneficial.